In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
train_df = pd.read_csv("Train.csv")
train_df.head()

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,target
0,Id_qt2qn56050,1624367008,pc1,-84.498750,-14.586875,-49.855625,16.457,35076.0,23.0,20.0,...,13.86,0.62,1013.6,2.39,0.95,4.0,16.093,0.38979,Residential,51500000.0
1,Id_f1zf07cwb6,1624372465,pc3,-86.818125,-11.982500,-54.838125,16.674,42704.0,24.0,20.0,...,14.52,0.59,1013.5,2.29,0.82,3.0,16.093,3.78322,Residential,50300000.0
2,Id_uoxdz7di5b,1624371871,pc3,-101.916250,-13.257500,-71.241250,9.392,36017.0,22.0,20.0,...,14.71,0.61,1013.8,2.13,0.82,3.0,16.093,2.08425,Residential,16500000.0
3,Id_hzn8nj29gu,1624542066,pc1,-95.598750,-14.696250,-61.660000,10.726,95955.0,17.0,20.0,...,13.43,0.71,1017.9,1.97,0.99,3.0,16.093,8.80702,Avenue,46400000.0
4,Id_2kfrccrfiw,1624542798,pc1,-97.735000,-14.867500,-63.170625,10.889,83062.0,19.0,20.0,...,13.46,0.71,1017.9,1.97,0.99,3.0,16.093,2.21843,Park,50200000.0


In [3]:
train_df.shape

(34274, 41)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34274 entries, 0 to 34273
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            34274 non-null  object 
 1   timestamp                     34274 non-null  int64  
 2   device                        34274 non-null  object 
 3   PCell_RSRP_max                34274 non-null  float64
 4   PCell_RSRQ_max                34274 non-null  float64
 5   PCell_RSSI_max                34274 non-null  float64
 6   PCell_SNR_max                 34274 non-null  float64
 7   PCell_Downlink_Num_RBs        34274 non-null  float64
 8   PCell_Downlink_Average_MCS    34274 non-null  float64
 9   PCell_Downlink_bandwidth_MHz  33207 non-null  float64
 10  PCell_Cell_Identity           33207 non-null  float64
 11  PCell_freq_MHz                34274 non-null  float64
 12  SCell_RSRP_max                19012 non-null  float64
 13  S

In [5]:
test_df = pd.read_csv("Test.csv")
test_df.head()

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area
0,Id_ln0e0hfrgx,1624374179,pc2,-97.121875,-12.923750,-87.554375,13.293,39276.0,24.0,20,...,23.05,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.07702,Avenue
1,Id_svf7nz9fxv,1624366264,pc4,-65.202500,-9.944375,-35.501875,21.823,39566.0,25.0,20,...,21.56,13.92,0.62,1013.6,2.42,0.95,4.0,16.093,0.66000,Park
2,Id_ww2mh07gwj,1624375795,pc2,-79.298125,-17.468750,-74.823750,24.114,41900.0,27.0,20,...,22.93,14.65,0.60,1013.5,2.26,0.85,3.0,16.093,2.49330,Park
3,Id_v88r4y03ww,1624374065,pc2,-110.128125,-13.513750,-92.566875,6.282,76310.0,18.0,20,...,22.95,14.45,0.59,1013.6,2.27,0.82,3.0,16.093,5.74454,Highway
4,Id_2u4y4kzglh,1624374361,pc2,-117.451250,-18.753125,-93.048125,-5.767,44887.0,7.0,20,...,23.05,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.48651,Avenue


In [6]:
test_df.shape

(18243, 40)

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18243 entries, 0 to 18242
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            18243 non-null  object 
 1   timestamp                     18243 non-null  int64  
 2   device                        18243 non-null  object 
 3   PCell_RSRP_max                18243 non-null  float64
 4   PCell_RSRQ_max                18243 non-null  float64
 5   PCell_RSSI_max                18243 non-null  float64
 6   PCell_SNR_max                 18243 non-null  float64
 7   PCell_Downlink_Num_RBs        18242 non-null  float64
 8   PCell_Downlink_Average_MCS    18242 non-null  float64
 9   PCell_Downlink_bandwidth_MHz  18243 non-null  int64  
 10  PCell_Cell_Identity           18243 non-null  float64
 11  PCell_freq_MHz                18243 non-null  float64
 12  SCell_RSRP_max                9628 non-null   float64
 13  S

In [8]:
train_df.isnull().sum()

id                                  0
timestamp                           0
device                              0
PCell_RSRP_max                      0
PCell_RSRQ_max                      0
PCell_RSSI_max                      0
PCell_SNR_max                       0
PCell_Downlink_Num_RBs              0
PCell_Downlink_Average_MCS          0
PCell_Downlink_bandwidth_MHz     1067
PCell_Cell_Identity              1067
PCell_freq_MHz                      0
SCell_RSRP_max                  15262
SCell_RSRQ_max                  15262
SCell_RSSI_max                  15262
SCell_SNR_max                   15262
SCell_Downlink_Num_RBs          15939
SCell_Downlink_Average_MCS      15939
SCell_Downlink_bandwidth_MHz    20849
SCell_Cell_Identity             20849
SCell_freq_MHz                  15262
operator                            0
Latitude                            0
Longitude                           0
Altitude                            7
speed_kmh                           0
COG         

In [9]:
test_df.isnull().sum()

id                                  0
timestamp                           0
device                              0
PCell_RSRP_max                      0
PCell_RSRQ_max                      0
PCell_RSSI_max                      0
PCell_SNR_max                       0
PCell_Downlink_Num_RBs              1
PCell_Downlink_Average_MCS          1
PCell_Downlink_bandwidth_MHz        0
PCell_Cell_Identity                 0
PCell_freq_MHz                      0
SCell_RSRP_max                   8615
SCell_RSRQ_max                   8615
SCell_RSSI_max                   8615
SCell_SNR_max                    8615
SCell_Downlink_Num_RBs           9350
SCell_Downlink_Average_MCS       9350
SCell_Downlink_bandwidth_MHz    13383
SCell_Cell_Identity             13383
SCell_freq_MHz                   8615
operator                            0
Latitude                            0
Longitude                           0
Altitude                            5
speed_kmh                           0
COG         

In [10]:
train_df[['SCell_Downlink_bandwidth_MHz']].value_counts()

SCell_Downlink_bandwidth_MHz
20.0                            13419
15.0                                6
dtype: int64

In [11]:
train_df[['SCell_freq_MHz']].value_counts()

SCell_freq_MHz
2600.0            17608
1800.0             1402
2000.0                2
dtype: int64

In [12]:
test_df[['SCell_freq_MHz']].value_counts()

SCell_freq_MHz
2600.0            9326
1800.0             302
dtype: int64

In [13]:
prep = train_df[['SCell_RSRP_max', 'SCell_RSRQ_max', 'SCell_RSSI_max', 'SCell_SNR_max', 'SCell_Downlink_Num_RBs', 'SCell_Downlink_Average_MCS', 'SCell_Downlink_bandwidth_MHz', 'SCell_Cell_Identity', 'SCell_freq_MHz']]
prep

,SCell_RSRP_max,SCell_RSRQ_max,SCell_RSSI_max,SCell_SNR_max,SCell_Downlink_Num_RBs,SCell_Downlink_Average_MCS,SCell_Downlink_bandwidth_MHz,SCell_Cell_Identity,SCell_freq_MHz
0,-87.075625,-10.159375,-56.209375,17.690,36756.0,19.0,NaN,NaN,2600.0
1,-93.598750,-12.158750,-62.156875,14.863,48372.0,14.0,20.0,2568967.0,2600.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
34269,-78.490625,-10.149375,-45.698125,20.326,94884.0,26.0,20.0,34007042.0,2600.0
34270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34271,-86.400000,-12.051250,-52.973125,13.028,41810.0,18.0,20.0,2826246.0,2600.0
34272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
train_df['PCell_Downlink_bandwidth_MHz'].median()

20.0

In [15]:
train_df['PCell_Downlink_bandwidth_MHz'].value_counts().to_frame()

,PCell_Downlink_bandwidth_MHz
20.0,28278
15.0,4849
5.0,80


In [16]:
prep.describe()

,SCell_RSRP_max,SCell_RSRQ_max,SCell_RSSI_max,SCell_SNR_max,SCell_Downlink_Num_RBs,SCell_Downlink_Average_MCS,SCell_Downlink_bandwidth_MHz,SCell_Cell_Identity,SCell_freq_MHz
count,19012.000000,19012.000000,19012.000000,19012.000000,18335.000000,18335.000000,13425.000000,1.342500e+04,19012.000000
mean,-89.184523,-11.529212,-57.416138,12.881317,70689.155659,18.759313,19.997765,2.472098e+07,2540.942563
std,9.508438,2.897617,8.681115,7.225271,23630.377794,5.115631,0.105684,1.175865e+07,209.159859
min,-174.971875,-30.000000,-86.156250,-16.900000,1.000000,0.000000,15.000000,2.568967e+06,1800.000000
25%,-95.137751,-12.273247,-63.003906,8.624000,48008.000000,15.000000,20.000000,2.635136e+07,2600.000000
50%,-89.544688,-10.496250,-58.248125,14.036500,79032.000000,19.000000,20.000000,2.660711e+07,2600.000000
75%,-82.386215,-9.778125,-51.104323,17.833000,92746.000000,22.000000,20.000000,3.082829e+07,2600.000000
max,-55.193125,-6.765000,-15.714375,28.404000,99388.000000,31.000000,20.000000,5.184257e+07,2600.000000


In [17]:
train_df['Traffic Jam Factor'].value_counts().to_frame()

,Traffic Jam Factor
3.14159,4202
0.00000,793
0.28864,198
8.49000,196
2.89119,190
...,...
2.64109,1
3.97126,1
0.67157,1
1.07558,1


In [18]:
# using the highest frequency to replace the missing values in training data

train_df['PCell_Downlink_bandwidth_MHz'].fillna(train_df['PCell_Downlink_bandwidth_MHz'].value_counts().index[0], inplace = True) 
train_df['SCell_Downlink_bandwidth_MHz'].fillna(train_df['SCell_Downlink_bandwidth_MHz'].value_counts().index[0], inplace = True) 
train_df['SCell_freq_MHz'].fillna(train_df['SCell_freq_MHz'].value_counts().index[0], inplace = True)

In [19]:
# using the highest frequency to replace the missing values in test data

test_df['SCell_Downlink_bandwidth_MHz'].fillna(test_df['SCell_Downlink_bandwidth_MHz'].value_counts().index[0], inplace = True) 
test_df['SCell_freq_MHz'].fillna(test_df['SCell_freq_MHz'].value_counts().index[0], inplace = True)

In [20]:
# using the median to replace the missing values in training data

train_df['SCell_RSRP_max'].fillna(train_df['SCell_RSRP_max'].median(), inplace = True) 
train_df['SCell_RSRQ_max'].fillna(train_df['SCell_RSRQ_max'].median(), inplace = True)
train_df['SCell_RSSI_max'].fillna(train_df['SCell_RSSI_max'].median(), inplace = True) 
train_df['SCell_SNR_max'].fillna(train_df['SCell_SNR_max'].median(), inplace = True)
train_df['SCell_Downlink_Num_RBs'].fillna(train_df['SCell_Downlink_Num_RBs'].median(), inplace = True) 
train_df['SCell_Downlink_Average_MCS'].fillna(train_df['SCell_Downlink_Average_MCS'].median(), inplace = True)
train_df['SCell_Cell_Identity'].fillna(train_df['SCell_Cell_Identity'].median(), inplace = True) 
train_df['Traffic Jam Factor'].fillna(train_df['Traffic Jam Factor'].median(), inplace = True)
train_df['PCell_Cell_Identity'].fillna(train_df['PCell_Cell_Identity'].median(), inplace = True)

In [21]:
# using the median to replace the missing values in test data

test_df['SCell_RSRP_max'].fillna(test_df['SCell_RSRP_max'].median(), inplace = True) 
test_df['SCell_RSRQ_max'].fillna(test_df['SCell_RSRQ_max'].median(), inplace = True)
test_df['SCell_RSSI_max'].fillna(test_df['SCell_RSSI_max'].median(), inplace = True) 
test_df['SCell_SNR_max'].fillna(test_df['SCell_SNR_max'].median(), inplace = True)
test_df['SCell_Downlink_Num_RBs'].fillna(test_df['SCell_Downlink_Num_RBs'].median(), inplace = True) 
test_df['SCell_Downlink_Average_MCS'].fillna(test_df['SCell_Downlink_Average_MCS'].median(), inplace = True)
test_df['SCell_Cell_Identity'].fillna(test_df['SCell_Cell_Identity'].median(), inplace = True) 
test_df['Traffic Jam Factor'].fillna(test_df['Traffic Jam Factor'].median(), inplace = True)

In [22]:
test_df.dropna(subset=['PCell_Downlink_Num_RBs'], inplace=True)
test_df.dropna(subset=['PCell_Downlink_Average_MCS'], inplace=True)

In [23]:
train_df.isnull().sum()

id                              0
timestamp                       0
device                          0
PCell_RSRP_max                  0
PCell_RSRQ_max                  0
PCell_RSSI_max                  0
PCell_SNR_max                   0
PCell_Downlink_Num_RBs          0
PCell_Downlink_Average_MCS      0
PCell_Downlink_bandwidth_MHz    0
PCell_Cell_Identity             0
PCell_freq_MHz                  0
SCell_RSRP_max                  0
SCell_RSRQ_max                  0
SCell_RSSI_max                  0
SCell_SNR_max                   0
SCell_Downlink_Num_RBs          0
SCell_Downlink_Average_MCS      0
SCell_Downlink_bandwidth_MHz    0
SCell_Cell_Identity             0
SCell_freq_MHz                  0
operator                        0
Latitude                        0
Longitude                       0
Altitude                        7
speed_kmh                       0
COG                             0
precipIntensity                 0
precipProbability               0
temperature   

In [24]:
train_df.dropna(subset=['Altitude'], inplace=True)
test_df.dropna(subset=['Altitude'], inplace=True)

In [25]:
test_df.isnull().sum()

id                              0
timestamp                       0
device                          0
PCell_RSRP_max                  0
PCell_RSRQ_max                  0
PCell_RSSI_max                  0
PCell_SNR_max                   0
PCell_Downlink_Num_RBs          0
PCell_Downlink_Average_MCS      0
PCell_Downlink_bandwidth_MHz    0
PCell_Cell_Identity             0
PCell_freq_MHz                  0
SCell_RSRP_max                  0
SCell_RSRQ_max                  0
SCell_RSSI_max                  0
SCell_SNR_max                   0
SCell_Downlink_Num_RBs          0
SCell_Downlink_Average_MCS      0
SCell_Downlink_bandwidth_MHz    0
SCell_Cell_Identity             0
SCell_freq_MHz                  0
operator                        0
Latitude                        0
Longitude                       0
Altitude                        0
speed_kmh                       0
COG                             0
precipIntensity                 0
precipProbability               0
temperature   

In [26]:
test_inputs = test_df.copy()

In [27]:
train_df.drop(['id'], axis = 1, inplace=True)
train_df.drop(['device'], axis = 1, inplace=True)
test_inputs.drop(['id'], axis = 1, inplace=True)
test_inputs.drop(['device'], axis = 1, inplace=True)

In [28]:
x = train_df.drop(columns=['target'])
y = train_df['target']

In [29]:
# Create an instance of the OrdinalEncoder class
encoder = OrdinalEncoder()

# Fit the encoder to the training data
encoder.fit(x[['area']])

# Transform the training data
x['area'] = encoder.transform(x[['area']])

# Transform the test data using the same encoder instance
test_inputs['area'] = encoder.transform(test_inputs[['area']])


In [30]:
x.head()

,timestamp,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,PCell_Cell_Identity,PCell_freq_MHz,...,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area
0,1624367008,-84.498750,-14.586875,-49.855625,16.457,35076.0,23.0,20.0,26368258.0,1800.0,...,21.56,13.86,0.62,1013.6,2.39,0.95,4.0,16.093,0.38979,3.0
1,1624372465,-86.818125,-11.982500,-54.838125,16.674,42704.0,24.0,20.0,2568970.0,1800.0,...,22.98,14.52,0.59,1013.5,2.29,0.82,3.0,16.093,3.78322,3.0
2,1624371871,-101.916250,-13.257500,-71.241250,9.392,36017.0,22.0,20.0,2824458.0,1800.0,...,22.57,14.71,0.61,1013.8,2.13,0.82,3.0,16.093,2.08425,3.0
3,1624542066,-95.598750,-14.696250,-61.660000,10.726,95955.0,17.0,20.0,26367489.0,1800.0,...,18.77,13.43,0.71,1017.9,1.97,0.99,3.0,16.093,8.80702,0.0
4,1624542798,-97.735000,-14.867500,-63.170625,10.889,83062.0,19.0,20.0,26369024.0,1800.0,...,18.79,13.46,0.71,1017.9,1.97,0.99,3.0,16.093,2.21843,2.0


In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 0)

In [32]:
# Create algorithm
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
val_predictions = rf.predict(x_test)
rf.score(x_test, y_test)

0.9443712319365262

In [33]:
# Test
test_predictions = rf.predict(test_inputs)

In [34]:
rmse = mean_squared_error(y_test, val_predictions, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")

Root Mean Squared Error = 10.0 Mbit/s


In [35]:
# Set parameters for XGBoost model
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror'
}

# Train XGBoost model
model = xgb.XGBRegressor(**params)
model.fit(x_train, y_train)

# Make predictions on test set
y_pred = model.predict(x_test)

In [36]:
model.score(x_test, y_test)

0.9155117078650421

In [37]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")

Root Mean Squared Error = 12.4 Mbit/s


In [38]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter space
params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0.1, 0.5, 1],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7]
}

# Create a random search object
xgb_random = RandomizedSearchCV(estimator = xgb.XGBRegressor(),
                                param_distributions = params,
                                n_iter = 100,
                                cv = 3,
                                verbose=2,
                                random_state=42,
                                n_jobs = -1)

# Fit the random search object to the data
xgb_random.fit(x_train, y_train)

# Print the best hyperparameters
print(xgb_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'subsample': 0.7, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.5}


In [39]:
predict = xgb_random.predict(x_test)
xgb_random.score(x_test, y_test)

0.9471937486841612

In [40]:
rmse = mean_squared_error(y_test, predict, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")

Root Mean Squared Error = 9.77 Mbit/s


In [41]:
test_predict = xgb_random.predict(test_inputs)

In [42]:
predictions_df = pd.DataFrame({'id': test_df.id, 'target': test_predict})
predictions_df.to_csv("BenchmarkSubmission.csv", index = False)
predictions_df.head()

,id,target
0,Id_ln0e0hfrgx,38107868.0
1,Id_svf7nz9fxv,64090792.0
2,Id_ww2mh07gwj,80069144.0
3,Id_v88r4y03ww,37238304.0
4,Id_2u4y4kzglh,25530762.0
